In [ ]:
import sqlite3
from datetime import datetime
import os

class ExpenseTracker:
    def __init__(self):
        self.db_name = "expenses.db"
        self.init_db()
        
    def init_db(self):
        """Initialize database with required tables"""
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        # Create expenses table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS expenses (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                amount REAL NOT NULL,
                category TEXT NOT NULL,
                description TEXT,
                date TEXT NOT NULL
            )
        ''')
        
        # Create categories table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS categories (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT UNIQUE NOT NULL
            )
        ''')
        
        # Insert default categories if they don't exist
        default_categories = ['Food', 'Transport', 'Entertainment', 'Utilities', 'Rent', 'Healthcare', 'Other']
        for category in default_categories:
            cursor.execute('INSERT OR IGNORE INTO categories (name) VALUES (?)', (category,))
        
        conn.commit()
        conn.close()
    
    def add_expense(self, amount, category, description=""):
        """Add a new expense to the database"""
        if amount <= 0:
            print("Amount must be positive!")
            return False
            
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        # Check if category exists
        cursor.execute('SELECT id FROM categories WHERE name = ?', (category,))
        if not cursor.fetchone():
            print(f"Category '{category}' doesn't exist!")
            conn.close()
            return False
        
        # Insert expense
        date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cursor.execute(
            'INSERT INTO expenses (amount, category, description, date) VALUES (?, ?, ?, ?)',
            (amount, category, description, date)
        )
        
        conn.commit()
        conn.close()
        print("Expense added successfully!")
        return True
    
    def view_expenses(self, period="all", category=None):
        """View expenses based on period and category filters"""
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        query = 'SELECT date, category, amount, description FROM expenses'
        params = []
        
        # Apply filters
        if period != "all" or category:
            conditions = []
            
            if period == "today":
                conditions.append("date(date) = date('now')")
            elif period == "week":
                conditions.append("date(date) >= date('now', '-7 days')")
            elif period == "month":
                conditions.append("date(date) >= date('now', '-30 days')")
            
            if category:
                conditions.append("category = ?")
                params.append(category)
            
            if conditions:
                query += " WHERE " + " AND ".join(conditions)
        
        query += " ORDER BY date DESC"
        cursor.execute(query, params)
        expenses = cursor.fetchall()
        
        if not expenses:
            print("No expenses found!")
            conn.close()
            return
        
        # Display expenses
        total = 0
        print("\n{:<19} {:<15} {:<10} {:<20}".format("Date", "Category", "Amount", "Description"))
        print("-" * 65)
        for expense in expenses:
            date, category, amount, description = expense
            total += amount
            print("{:<19} {:<15} ${:<9.2f} {:<20}".format(
                date, category, amount, description[:20] + "..." if len(description) > 20 else description
            ))
        
        print("-" * 65)
        print("{:<35} ${:<10.2f}".format("TOTAL", total))
        conn.close()
    
    def view_summary(self, period="month"):
        """View summary of expenses by category"""
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        if period == "today":
            date_condition = "AND date(date) = date('now')"
        elif period == "week":
            date_condition = "AND date(date) >= date('now', '-7 days')"
        else:  # month
            date_condition = "AND date(date) >= date('now', '-30 days')"
        
        cursor.execute(f'''
            SELECT category, SUM(amount) 
            FROM expenses 
            WHERE 1=1 {date_condition}
            GROUP BY category 
            ORDER BY SUM(amount) DESC
        ''')
        
        summary = cursor.fetchall()
        
        if not summary:
            print("No expenses found for the selected period!")
            conn.close()
            return
        
        total = sum(amount for _, amount in summary)
        
        print(f"\nExpense Summary ({period}):")
        print("{:<15} {:<10} {:<15}".format("Category", "Amount", "Percentage"))
        print("-" * 45)
        for category, amount in summary:
            percentage = (amount / total) * 100 if total > 0 else 0
            print("{:<15} ${:<9.2f} {:<14.1f}%".format(category, amount, percentage))
        
        print("-" * 45)
        print("{:<15} ${:<9.2f}".format("TOTAL", total))
        conn.close()
    
    def add_category(self, category_name):
        """Add a new category"""
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        try:
            cursor.execute('INSERT INTO categories (name) VALUES (?)', (category_name,))
            conn.commit()
            print(f"Category '{category_name}' added successfully!")
        except sqlite3.IntegrityError:
            print(f"Category '{category_name}' already exists!")
        finally:
            conn.close()
    
    def view_categories(self):
        """View all available categories"""
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute('SELECT name FROM categories ORDER BY name')
        categories = cursor.fetchall()
        
        print("\nAvailable Categories:")
        print("-" * 20)
        for i, (category,) in enumerate(categories, 1):
            print(f"{i}. {category}")
        
        conn.close()

def main():
    tracker = ExpenseTracker()
    
    while True:
        print("\n" + "=" * 50)
        print("EXPENSE TRACKER")
        print("=" * 50)
        print("1. Add Expense")
        print("2. View Expenses")
        print("3. View Summary")
        print("4. Add Category")
        print("5. View Categories")
        print("6. Exit")
        
        choice = input("\nEnter your choice (1-6): ").strip()
        
        if choice == "1":
            try:
                amount = float(input("Enter amount: $"))
                tracker.view_categories()
                category = input("Enter category: ").strip()
                description = input("Enter description (optional): ").strip()
                tracker.add_expense(amount, category, description)
            except ValueError:
                print("Invalid amount! Please enter a number.")
        
        elif choice == "2":
            print("\nView Expenses:")
            print("a. All expenses")
            print("b. Today's expenses")
            print("c. This week's expenses")
            print("d. This month's expenses")
            period_choice = input("Enter your choice (a-d): ").strip().lower()
            
            period_map = {
                'a': 'all',
                'b': 'today',
                'c': 'week',
                'd': 'month'
            }
            
            period = period_map.get(period_choice, 'all')
            
            filter_by_category = input("Filter by category? (y/n): ").strip().lower()
            category = None
            if filter_by_category == 'y':
                tracker.view_categories()
                category = input("Enter category name: ").strip()
            
            tracker.view_expenses(period, category)
        
        elif choice == "3":
            print("\nView Summary:")
            print("a. Today")
            print("b. This week")
            print("c. This month")
            period_choice = input("Enter your choice (a-c): ").strip().lower()
            
            period_map = {
                'a': 'today',
                'b': 'week',
                'c': 'month'
            }
            
            period = period_map.get(period_choice, 'month')
            tracker.view_summary(period)
        
        elif choice == "4":
            category = input("Enter new category name: ").strip()
            if category:
                tracker.add_category(category)
            else:
                print("Category name cannot be empty!")
        
        elif choice == "5":
            tracker.view_categories()
        
        elif choice == "6":
            print("Goodbye!")
            break
        
        else:
            print("Invalid choice! Please try again.")
        
        input("\nPress Enter to continue...")

if __name__ == "__main__":
    main()


EXPENSE TRACKER
1. Add Expense
2. View Expenses
3. View Summary
4. Add Category
5. View Categories
6. Exit

View Summary:
a. Today
b. This week
c. This month
No expenses found for the selected period!
